In [1]:
!pip install tensorflow tensorflow-datasets git+https://github.com/tensorflow/docs --quiet


  Preparing metadata (setup.py) ... done


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from tensorflow.keras import layers, regularizers
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots


In [3]:
gz = tf.keras.utils.get_file(
    'HIGGS.csv.gz',
    'http://mlphysics.ics.uci.edu/data/higgs/HIGGS.csv.gz'
)

FEATURES = 28
ds = tf.data.experimental.CsvDataset(gz, [float(),] * (FEATURES + 1), compression_type="GZIP")


2816407858/2816407858 ━━━━━━━━━━━━━━━━━━━━ 231s 0us/step


In [4]:
def pack_row(*row):
    label = row[0]
    features = tf.stack(row[1:], axis=1)
    return features, label

packed_ds = ds.batch(10000).map(pack_row).unbatch()


In [5]:
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(1e4)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE

validate_ds = packed_ds.take(N_VALIDATION).cache()
train_ds = packed_ds.skip(N_VALIDATION).take(N_TRAIN).cache()

train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)
validate_ds = validate_ds.batch(BATCH_SIZE)


In [6]:
def get_callbacks(name):
    return [
        tfdocs.modeling.EpochDots(),
        tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
    ]

def compile_and_fit(model, name, max_epochs=10000):
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    history = model.fit(
        train_ds,
        steps_per_epoch=STEPS_PER_EPOCH,
        epochs=max_epochs,
        validation_data=validate_ds,
        callbacks=get_callbacks(name),
        verbose=0
    )
    return history


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers

# 📌 Fix 1: Use `Input(shape=())` instead of passing `input_shape` in Dense layers
tiny_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28,)),  # Input layer with 28 features
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# 📌 Fix 2: Ensure dataset is repeated to prevent "ran out of data" error
train_ds = train_ds.repeat()  # Allows infinite iterations

# 📌 Fix 3: Explicitly set `mode='min'` for EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_binary_crossentropy',
    mode='min',
    patience=5  # Stops if validation loss doesn't improve for 5 epochs
)

# Compile the model
tiny_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history_tiny = tiny_model.fit(
    train_ds,
    validation_data=validate_ds,
    epochs=100,
    steps_per_epoch=STEPS_PER_EPOCH,  # Prevents "ran out of data" issue
    callbacks=[early_stopping]
)


Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4943 - loss: 0.8092 - val_accuracy: 0.5270 - val_loss: 0.7465
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5049 - loss: 0.7551 - val_accuracy: 0.5310 - val_loss: 0.7239
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5048 - loss: 0.7314 - val_accuracy: 0.5250 - val_loss: 0.7097
Epoch 4/100
 1/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5580 - loss: 0.7056

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_binary_crossentropy` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5191 - loss: 0.7097 - val_accuracy: 0.5280 - val_loss: 0.7012
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5205 - loss: 0.7009 - val_accuracy: 0.5420 - val_loss: 0.6943
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5346 - loss: 0.6931 - val_accuracy: 0.5470 - val_loss: 0.6893
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5471 - loss: 0.6858 - val_accuracy: 0.5510 - val_loss: 0.6847
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5528 - loss: 0.6830 - val_accuracy: 0.5470 - val_loss: 0.6808
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5575 - loss: 0.6797 - val_accuracy: 0.5470 - val_loss: 0.6778
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5789 - loss: 0.6703 - val_accuracy: 0.5490 - val_loss: 0.6747
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5792 - loss: 0.6709 - val_accuracy: 0.5540 - val_loss: 0

In [9]:
# Evaluate the model on the validation dataset
test_loss, test_accuracy = tiny_model.evaluate(validate_ds)

print(f"✅ Test Accuracy: {test_accuracy:.4f}")
print(f"✅ Test Loss: {test_loss:.4f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6660 - loss: 0.6172
✅ Test Accuracy: 0.6580
✅ Test Loss: 0.6245


In [10]:
# Example test input (28 features)
import numpy as np
sample_input = np.random.rand(1, 28)  # Generate a random test input

# Predict
prediction = tiny_model.predict(sample_input)

print(f"🔮 Prediction: {prediction[0][0]:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
🔮 Prediction: 0.4545


In [11]:
# Save the model
tiny_model.save("tiny_model.keras")

# Load the model (if needed)
reloaded_model = tf.keras.models.load_model("tiny_model.keras")
print("✅ Model reloaded successfully!")


✅ Model reloaded successfully!
